In [ ]:
import tensorflow as tf
import math
from tensorflow.examples.tutorials.mnist import input_data as mnist_data
import numpy as np
print("Tensorflow version " + tf.__version__)
tf.set_random_seed(0)

In [ ]:
mnist = mnist_data.read_data_sets("data", one_hot=True, reshape=False, validation_size=0)

In [ ]:
x_train = np.array(mnist.train.images)
x_test = np.array(mnist.test.images)

y_train = np.array(mnist.train.labels) 
y_test = np.array(mnist.test.labels)

In [ ]:
X = tf.placeholder(tf.float32, [None, 28, 28, 1])
# correct answers will go here
Y_ = tf.placeholder(tf.float32, [None, 10])
# variable learning rate
lr = tf.placeholder(tf.float32)

In [ ]:
K = 4  # first convolutional layer output depth
L = 8  # second convolutional layer output depth
M = 12  # third convolutional layer
N = 200  # fully connected layer

In [ ]:
#W1 = tf.Variable(tf.truncated_normal([5, 5, 1, K], stddev=0.1))  # 5x5 patch, 1 input channel, K output channels
#B1 = tf.Variable(tf.ones([K])/10)
#W2 = tf.Variable(tf.truncated_normal([5, 5, K, L], stddev=0.1))
#B2 = tf.Variable(tf.ones([L])/10)
#W3 = tf.Variable(tf.truncated_normal([4, 4, L, M], stddev=0.1))
#B3 = tf.Variable(tf.ones([M])/10)

#W4 = tf.Variable(tf.truncated_normal([7 * 7 * M, N], stddev=0.1))
#B4 = tf.Variable(tf.ones([N])/10)
#W5 = tf.Variable(tf.truncated_normal([N, 10], stddev=0.1))
#B5 = tf.Variable(tf.ones([10])/10)

#Using Xavier Initializatiion
W1 = tf.get_variable("W1", [5, 5, 1, K], initializer=tf.contrib.layers.xavier_initializer(seed=1)) # 5x5 patch, 1 input channel, 4 output channels
B1 = tf.get_variable("B1", [K], initializer=tf.zeros_initializer())
W2 = tf.get_variable("W2", [5, 5, K, L], initializer=tf.contrib.layers.xavier_initializer(seed=10))
B2 = tf.get_variable("B2", [L], initializer=tf.zeros_initializer())
W3 = tf.get_variable("W3", [4, 4, L, M], initializer=tf.contrib.layers.xavier_initializer(seed=0))
B3 = tf.get_variable("B3", [M], initializer=tf.zeros_initializer())

W4 = tf.get_variable("W4", [7*7*M, N], initializer=tf.contrib.layers.xavier_initializer(seed=11))
B4 = tf.get_variable("B4", [N], initializer=tf.zeros_initializer())
W5 = tf.get_variable("W5", [N, 10], initializer=tf.contrib.layers.xavier_initializer())
B5 = tf.get_variable("B5", [10], initializer=tf.zeros_initializer())

parameters = {"W1": W1,
              "B1": B1,
              "W2": W2,
              "B2": B2,
              "W3": W3,
              "B3": B3,
              "W4": W4,
              "B4": B4,
              "W5": W5,
              "B5": B5}

In [ ]:
# The model
stride = 1  # output is 28x28
Y1 = tf.nn.relu(tf.nn.conv2d(X, W1, strides=[1, stride, stride, 1], padding='SAME') + B1)
stride = 2  # output is 14x14
Y2 = tf.nn.relu(tf.nn.conv2d(Y1, W2, strides=[1, stride, stride, 1], padding='SAME') + B2)
stride = 2  # output is 7x7
Y3 = tf.nn.relu(tf.nn.conv2d(Y2, W3, strides=[1, stride, stride, 1], padding='SAME') + B3)

# reshape the output from the third convolution for the fully connected layer
#YY = tf.reshape(Y3, shape=[-1, 7 * 7 * M])
YY = tf.contrib.layers.flatten(Y3)
Y4 = tf.nn.relu(tf.matmul(YY, W4) + B4)
Ylogits = tf.matmul(Y4, W5) + B5
Y = tf.nn.softmax(Ylogits)

In [ ]:
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=Ylogits, labels=Y_)
cross_entropy = tf.reduce_mean(cross_entropy)*100# accuracy of the trained model, between 0 (worst) and 1 (best)


In [ ]:
# training step, the learning rate is a placeholder
train_step = tf.train.AdamOptimizer(lr).minimize(cross_entropy)
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(100):
        # learning rate decay
        max_learning_rate = 0.003
        min_learning_rate = 0.0001
        decay_speed = 2000.0
        learning_rate = min_learning_rate + (max_learning_rate - min_learning_rate) * math.exp(-epoch/decay_speed)
        for minibatch in range(600):
            epoch_cost = 0.
            batch_X, batch_Y = mnist.train.next_batch(100)
            _ , minibatch_cost = sess.run([train_step, cross_entropy], feed_dict={X: batch_X, Y_: batch_Y, lr: learning_rate})
            epoch_cost += minibatch_cost / 600
        if epoch % 10 == 0:
            print ("Cost after epoch %i: %f" % (epoch, epoch_cost))
    parameters = sess.run(parameters)
    #Identify Correct Predictions
    correct_prediction = tf.equal(tf.argmax(Y, 1), tf.argmax(Y_, 1))
    #Calculate Accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    #print ("Train Accuracy:", accuracy.eval({X: x_train, Y_: y_train}))
    print ("Test Accuracy:", accuracy.eval({X: x_test, Y_: y_test}))

In [ ]:
def forward_propagation_for_predict(X, parameters):
    # Retrieve the parameters from the dictionary "parameters" 
    W1 = parameters['W1']
    W2 = parameters['W2']
    W3 = parameters['W3']
    W4 = parameters['W4']
    W5 = parameters['W5']
    B1 = parameters['B1']
    B2 = parameters['B2']
    B3 = parameters['B3']
    B4 = parameters['B4']
    B5 = parameters['B5']
    
    Z1 = tf.nn.conv2d(X, W1, strides = [1,1,1,1], padding = "SAME")
    A1 = tf.nn.relu(Z1 + B1)
    Z2 = tf.nn.conv2d(A1, W2, strides = [1,2,2,1], padding = "SAME")
    A2 = tf.nn.relu(Z2 + B2)
    Z3 = tf.nn.conv2d(A2, W3, strides = [1,2,2,1], padding = "SAME")
    A3 = tf.nn.relu(Z3 + B3)    
    P1 = tf.contrib.layers.flatten(A3)
    Z4 = tf.add(tf.matmul(P1, W4), B4)
    A4 = tf.nn.relu(Z4)
    Z5 = tf.matmul(A4, W5) + B5
    return Z5

In [ ]:
def imageprepare(argv):
    """
    This function returns the pixel values.
    The imput is a png file location.
    """
    im = Image.open(argv).convert('L')
    width = float(im.size[0])
    height = float(im.size[1])
    newImage = Image.new('L', (28, 28), (255))  # creates white canvas of 28x28 pixels

    if width > height:  # check which dimension is bigger
        # Width is bigger. Width becomes 20 pixels.
        nheight = int(round((20.0 / width * height), 0))  # resize height according to ratio width
        if (nheight == 0):  # rare case but minimum is 1 pixel
            nheight = 1
            # resize and sharpen
        img = im.resize((20, nheight), Image.ANTIALIAS).filter(ImageFilter.SHARPEN)
        wtop = int(round(((28 - nheight) / 2), 0))  # calculate horizontal position
        newImage.paste(img, (4, wtop))  # paste resized image on white canvas
    else:
        # Height is bigger. Heigth becomes 20 pixels.
        nwidth = int(round((20.0 / height * width), 0))  # resize width according to ratio height
        if (nwidth == 0):  # rare case but minimum is 1 pixel
            nwidth = 1
            # resize and sharpen
        img = im.resize((nwidth, 20), Image.ANTIALIAS).filter(ImageFilter.SHARPEN)
        wleft = int(round(((28 - nwidth) / 2), 0))  # caculate vertical pozition
        newImage.paste(img, (wleft, 4))  # paste resized image on white canvas

    # newImage.save("sample.png

    tv = list(newImage.getdata())  # get pixel values

    # normalize pixels to 0 and 1. 0 is pure white, 1 is pure black.
    tva = [(255 - x) * 1.0 / 255.0 for x in tv]
    return tva

In [ ]:
#Prediction on Own Image

import scipy
from PIL import Image, ImageFilter
from scipy import ndimage
from matplotlib import pyplot as plt

#Image Processing to fit the algorithm
my_image_logic = [imageprepare('images/handwriting.png')]
my_image = np.transpose(my_image_logic)
newArr=[[0 for d in range(28)] for y in range(28)]
k = 0
for i in range(28):
    for j in range(28):
        newArr[i][j]=my_image_logic[0][k]
        k=k+1
plt.imshow(newArr, interpolation='nearest')
plt.show()
i = np.array(newArr)
i = i.reshape(1, 28, 28, 1)
W1 = tf.convert_to_tensor(parameters["W1"])
W2 = tf.convert_to_tensor(parameters["W2"])
W3 = tf.convert_to_tensor(parameters["W3"])
W4 = tf.convert_to_tensor(parameters["W4"])
W5 = tf.convert_to_tensor(parameters["W5"])
B1 = tf.convert_to_tensor(parameters["B1"])
B2 = tf.convert_to_tensor(parameters["B2"])
B3 = tf.convert_to_tensor(parameters["B3"])
B4 = tf.convert_to_tensor(parameters["B4"])
B5 = tf.convert_to_tensor(parameters["B5"])

    
params = {"W1": W1,
          "B1": B1,
          "W2": W2,
          "B2": B2,
          "W3": W3,
          "B3": B3,
          "W4": W4,
          "B4": B4,
          "W5": W5,
          "B5": B5}

x = tf.placeholder(tf.float32, [1, 28, 28, 1])

z5 = forward_propagation_for_predict(x, params)

p = tf.argmax(z5, 1)

sess = tf.Session()
my_image_prediction = sess.run(p, feed_dict = {x: i})

print("Your algorithm predicts: y = " + str(np.squeeze(my_image_prediction)))